In [1]:
!pip install kagglehub --upgrade

In [2]:
import kagglehub

In [3]:
# download dataset from kaggle

data_path = kagglehub.dataset_download("undefinenull/million-song-dataset-spotify-lastfm")

In [4]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# segregate 2 data's

data_path = Path(data_path)

songs_data_path = data_path / 'Music Info.csv'
users_data_path = data_path / 'User Listening History.csv'

In [6]:
# load the songs dataset

df_songs = pd.read_csv(songs_data_path)
df_songs.head()

track_id             name           artist  \
0  TRIOREW128F424EAF0   Mr. Brightside      The Killers   
1  TRRIVDJ128F429B0E8       Wonderwall            Oasis   
2  TROUVHL128F426C441  Come as You Are          Nirvana   
3  TRUEIND128F93038C4      Take Me Out  Franz Ferdinand   
4  TRLNZBD128F935E4D8            Creep        Radiohead   

                                 spotify_preview_url              spotify_id  \
0  https://p.scdn.co/mp3-preview/4d26180e6961fd46...  09ZQ5TmUG8TSL56n0knqrj   
1  https://p.scdn.co/mp3-preview/d012e536916c927b...  06UfBBDISthj1ZJAtX4xjj   
2  https://p.scdn.co/mp3-preview/a1c11bb1cb231031...  0keNu0t0tqsWtExGM3nT1D   
3  https://p.scdn.co/mp3-preview/399c401370438be4...  0ancVQ9wEcHVd0RrGICTE4   
4  https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...  01QoK9DA7VTeTSE3MNzp4I   

                                                tags genre  year  duration_ms  \
0  rock, alternative, indie, alternative_rock, in...   NaN  2004       222200   
1  rock, alternative, indie, pop, alternative_roc...   NaN  2006       258613   
2   rock, alternative, alternative_rock, 90s, grunge   RnB  1991       218920   
3  rock, alternative, indie, alternative_rock, in...   NaN  2004       237026   
4  rock, alternative, indie, alternative_rock, in...   RnB  2008       238640   

   danceability  ...  key  loudness  mode  speechiness  acousticness  \
0         0.355  ...    1    -4.360     1       0.0746      0.001190   
1         0.409  ...    2    -4.373     1       0.0336      0.000807   
2         0.508  ...    4    -5.783     0       0.0400      0.000175   
3         0.279  ...    9    -8.851     1       0.0371      0.000389   
4         0.515  ...    7    -9.935     1       0.0369      0.010200   

   instrumentalness  liveness  valence    tempo  time_signature  
0          0.000000    0.0971    0.240  148.114               4  
1          0.000000    0.2070    0.651  174.426               4  
2          0.000459    0.0878    0.543  120.012               4  
3          0.000655    0.1330    0.490  104.560               4  
4          0.000141    0.1290    0.104   91.841               4  

[5 rows x 21 columns]

In [7]:
# size of dataset

df_songs.shape

(50683, 21)

In [8]:
# data info

df_songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50683 entries, 0 to 50682
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   track_id             50683 non-null  object 
 1   name                 50683 non-null  object 
 2   artist               50683 non-null  object 
 3   spotify_preview_url  50683 non-null  object 
 4   spotify_id           50683 non-null  object 
 5   tags                 49556 non-null  object 
 6   genre                22348 non-null  object 
 7   year                 50683 non-null  int64  
 8   duration_ms          50683 non-null  int64  
 9   danceability         50683 non-null  float64
 10  energy               50683 non-null  float64
 11  key                  50683 non-null  int64  
 12  loudness             50683 non-null  float64
 13  mode                 50683 non-null  int64  
 14  speechiness          50683 non-null  float64
 15  acousticness         50683 non-null 

# Data Cleaning



1.   remove duplicates
2.   since genre has 56% values as missing, we will drop it
3.   assign missing values of tags as "no tags"
4.   convert columns to lowercase
5.   drop unique columns (like Name, Id, etc.) on which similarity can't be calculated





In [9]:
# reset the index

df_songs.reset_index(drop=True, inplace=True)

In [10]:
# duplicates in the data based on spotify_id

df_songs.duplicated(subset="spotify_id").sum()

np.int64(9)

In [11]:
# drop duplicates

df_songs.drop_duplicates(subset=["spotify_id", "year", "duration_ms"], inplace=True)

In [12]:
# check for duplicates

df_songs.duplicated(subset=["spotify_id", "year", "duration_ms"]).sum()

np.int64(0)

In [13]:
# remove columns not required for collaborative filtering

cols_to_remove = ['track_id', 'name', 'spotify_id', 'genre', 'spotify_preview_url']
df_content_filtering = df_songs.drop(columns=cols_to_remove)
df_content_filtering

artist  \
0                The Killers   
1                      Oasis   
2                    Nirvana   
3            Franz Ferdinand   
4                  Radiohead   
...                      ...   
50678            アンティック-珈琲店-   
50679                ACIDMAN   
50680               coldrain   
50681            アンティック-珈琲店-   
50682  9mm Parabellum Bullet   

                                                    tags  year  duration_ms  \
0      rock, alternative, indie, alternative_rock, in...  2004       222200   
1      rock, alternative, indie, pop, alternative_roc...  2006       258613   
2       rock, alternative, alternative_rock, 90s, grunge  1991       218920   
3      rock, alternative, indie, alternative_rock, in...  2004       237026   
4      rock, alternative, indie, alternative_rock, in...  2008       238640   
...                                                  ...   ...          ...   
50678                                                NaN  2008       273440   
50679            rock, alternative_rock, japanese, cover  2004       275133   
50680                    metal, metalcore, post_hardcore  2014       254826   
50681                                                NaN  2008       243293   
50682                                     rock, japanese  2008       131960   

       danceability  energy  key  loudness  mode  speechiness  acousticness  \
0             0.355   0.918    1    -4.360     1       0.0746      0.001190   
1             0.409   0.892    2    -4.373     1       0.0336      0.000807   
2             0.508   0.826    4    -5.783     0       0.0400      0.000175   
3             0.279   0.664    9    -8.851     1       0.0371      0.000389   
4             0.515   0.430    7    -9.935     1       0.0369      0.010200   
...             ...     ...  ...       ...   ...          ...           ...   
50678         0.438   0.933    6    -3.062     0       0.1650      0.003120   
50679         0.351   0.693    0    -6.811     1       0.1200      0.000940   
50680         0.434   0.975   10    -3.092     0       0.2680      0.000108   
50681         0.513   0.902    4    -3.914     0       0.0530      0.000715   
50682         0.190   0.953    8    -4.144     1       0.1310      0.000033   

       instrumentalness  liveness  valence    tempo  time_signature  
0              0.000000    0.0971    0.240  148.114               4  
1              0.000000    0.2070    0.651  174.426               4  
2              0.000459    0.0878    0.543  120.012               4  
3              0.000655    0.1330    0.490  104.560               4  
4              0.000141    0.1290    0.104   91.841               4  
...                 ...       ...      ...      ...             ...  
50678          0.000000    0.1300    0.421  166.956               4  
50679          0.000049    0.1920    0.450  200.350               4  
50680          0.001410    0.1630    0.282  158.025               4  
50681          0.001350    0.0571    0.618  109.923               4  
50682          0.051100    0.2030    0.138  136.475               4  

[50674 rows x 16 columns]

In [14]:
# check for missing values

df_content_filtering.isna().sum()

artist                 0
tags                1126
year                   0
duration_ms            0
danceability           0
energy                 0
key                    0
loudness               0
mode                   0
speechiness            0
acousticness           0
instrumentalness       0
liveness               0
valence                0
tempo                  0
time_signature         0
dtype: int64

In [15]:
# fill the tags column missing values with string "no tags"

df_content_filtering.fillna({"tags": "no_tags"}, inplace=True)

In [16]:
# check for missing values

df_content_filtering.isna().sum()

artist              0
tags                0
year                0
duration_ms         0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
dtype: int64

In [17]:
# artists names as lowercase

df_content_filtering["artist"] = df_content_filtering["artist"].str.lower()

In [18]:
# number of unique artists

df_songs.loc[:,'artist'].nunique()

8317

In [19]:
# number of unique year values

df_songs.loc[:,'year'].nunique()

75

In [20]:
# min & max of numerical columns

df_songs.select_dtypes(include="int").agg(["min", "max"])

year  duration_ms  key  mode  time_signature
min  1900         1439    0     0               0
max  2022      3816373   11     1               5

In [21]:
# value counts for the tags

(
    df_songs
    .loc[:,'tags']
    .str.lower()
    .str.split(',')
    .explode()
    .str.strip()
    .value_counts()
)

tags
rock            10681
indie            7284
electronic       6592
alternative      6271
pop              4650
                ...  
dark_ambient      602
japanese          489
polish            411
j_pop             213
russian           127
Name: count, Length: 100, dtype: int64

In [22]:
# tags whose value is greater than 1K

(
    df_songs
    .loc[:,'tags']
    .str.lower()
    .str.split(',')
    .explode()
    .str.strip()
    .value_counts()
    .loc[lambda ser: ser>=1000]
)

tags
rock            10681
indie            7284
electronic       6592
alternative      6271
pop              4650
                ...  
ska              1088
gothic_metal     1072
grindcore        1040
french           1018
nu_metal         1006
Name: count, Length: 85, dtype: int64

# Transform the data

In [23]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.1 MB/s eta 0:00:00


In [24]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from category_encoders.count import CountEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

In [25]:
df_content_filtering.head(3)

artist                                               tags  year  \
0  the killers  rock, alternative, indie, alternative_rock, in...  2004   
1        oasis  rock, alternative, indie, pop, alternative_roc...  2006   
2      nirvana   rock, alternative, alternative_rock, 90s, grunge  1991   

   duration_ms  danceability  energy  key  loudness  mode  speechiness  \
0       222200         0.355   0.918    1    -4.360     1       0.0746   
1       258613         0.409   0.892    2    -4.373     1       0.0336   
2       218920         0.508   0.826    4    -5.783     0       0.0400   

   acousticness  instrumentalness  liveness  valence    tempo  time_signature  
0      0.001190          0.000000    0.0971    0.240  148.114               4  
1      0.000807          0.000000    0.2070    0.651  174.426               4  
2      0.000175          0.000459    0.0878    0.543  120.012               4

In [26]:
df_content_filtering.shape

(50674, 16)

In [27]:
# columns to transform

frequency_encode_cols = ["year"]
ohe_cols = ["artist", "time_signature", "key"]
tfidf_cols = 'tags' # TfidfVectorizer expects a 1D text-like input — a Series, not a DataFrame
standard_scale_cols = ["duration_ms", "loudness", "tempo"]
min_max_scale_cols = ["danceability", "energy", "speechiness", "acousticness", "instrumentalness", "liveness", "valence"]

In [28]:
len(frequency_encode_cols + ohe_cols + standard_scale_cols + min_max_scale_cols)

14

In [29]:
# transform the data

transformer = ColumnTransformer(transformers=[
    ("frequency_encode", CountEncoder(normalize=True, return_df=True), frequency_encode_cols),
    ("ohe", OneHotEncoder(handle_unknown="ignore"), ohe_cols),
    ("tfidf", TfidfVectorizer(max_features=85), tfidf_cols),
    ("standard_scale", StandardScaler(), standard_scale_cols),
    ("min_max_scale", MinMaxScaler(), min_max_scale_cols)
], remainder="passthrough", n_jobs=-1, force_int_remainder_cols=False)

transformer

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('frequency_encode',
                                 CountEncoder(combine_min_nan_groups=True,
                                              normalize=True),
                                 ['year']),
                                ('ohe', OneHotEncoder(handle_unknown='ignore'),
                                 ['artist', 'time_signature', 'key']),
                                ('tfidf', TfidfVectorizer(max_features=85),
                                 'tags'),
                                ('standard_scale', StandardScaler(),
                                 ['duration_ms', 'loudness', 'tempo']),
                                ('min_max_scale', MinMaxScaler(),
                                 ['danceability', 'energy', 'speechiness',
                                  'acousticness', 'instrumentalness',
                                  'liveness', 'valence'])])

In [30]:
# fit the transformer

transformer.fit(df_content_filtering)

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('frequency_encode',
                                 CountEncoder(combine_min_nan_groups=True,
                                              normalize=True),
                                 ['year']),
                                ('ohe', OneHotEncoder(handle_unknown='ignore'),
                                 ['artist', 'time_signature', 'key']),
                                ('tfidf', TfidfVectorizer(max_features=85),
                                 'tags'),
                                ('standard_scale', StandardScaler(),
                                 ['duration_ms', 'loudness', 'tempo']),
                                ('min_max_scale', MinMaxScaler(),
                                 ['danceability', 'energy', 'speechiness',
                                  'acousticness', 'instrumentalness',
                                  'liveness', 'valence'])])

In [31]:
# transform the data

transformed_df = transformer.transform(df_content_filtering)

In [32]:
transformed_df.shape

(50674, 8431)

In [33]:
transformed_df # it's a sparse matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 907911 stored elements and shape (50674, 8431)>

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
# featch songs where artist in Shakira

df_songs.loc[df_songs['artist'] == "Shakira", :]

track_id                            name   artist  \
1025   TRLWDVU128F932B093              Whenever, Wherever  Shakira   
2068   TRILOWN128F426080F         Underneath Your Clothes  Shakira   
2205   TRXLMFJ12903CC06F7                        She Wolf  Shakira   
2469   TRSQAWU128F92EA20F                      La Tortura  Shakira   
3374   TRINUNP12903CD84D9                    Did It Again  Shakira   
4319   TROLUWR128F92E5858                  Te Dejo Madrid  Shakira   
4600   TRLNLES128F932DA8E                            Fool  Shakira   
4618   TRUIPZL12903CA0BFE                           Rules  Shakira   
6047   TRAAKDG128F42A0ECB                  Hips Don't Lie  Shakira   
6819   TRBAHID128F4278EAF               Objection (Tango)  Shakira   
7008   TRHEBDB128F92E9FC0                    Don't Bother  Shakira   
11905  TRADBSW128F933A7A8                            Loba  Shakira   
12306  TRYFVKK128F4240FE8                        Why Wait  Shakira   
12323  TROZGSI128F92EA0AE                        La Pared  Shakira   
16861  TRAFCNH12903CA072C                      Inevitable  Shakira   
16947  TRUGCOP128F4232463                       Si te vas  Shakira   
27044  TRSAGEK128F9333C7C               Lo Imprescindible  Shakira   
29948  TRUIESB128F427D70D                              Tú  Shakira   
32185  TROSDVD12903CB116F                       Mon Amour  Shakira   
38389  TRWUWRZ128F42ADA4A                Dreams for Plans  Shakira   
38421  TRNHCIP128F42ADA54         Costume Makes the Clown  Shakira   
42435  TRGJXTC128F4278ECC                Que Me Quedes Tu  Shakira   
42453  TRCYQFE128F42ADA40                     Animal City  Shakira   
42514  TRYTKQE128F42ADA50                    Your Embrace  Shakira   
42562  TRFNIOO128F42AD944             Donde Estas Corazon  Shakira   
43704  TRANQLL128F92F5263                 Poem to a Horse  Shakira   
43712  TRIXHNL128F1496B61                     Sale el Sol  Shakira   
43714  TRULZUZ128F427D869        Ready for the Good Times  Shakira   
43735  TRULHLZ128F42BB903                         Illegal  Shakira   
43891  TRDENYV128F4295BF9  Pies Descalzos, Sueños Blancos  Shakira   

                                     spotify_preview_url  \
1025   https://p.scdn.co/mp3-preview/09ddeb4ae33ee6e8...   
2068   https://p.scdn.co/mp3-preview/6c5a56058ce04371...   
2205   https://p.scdn.co/mp3-preview/4dc802fd3b06fcb5...   
2469   https://p.scdn.co/mp3-preview/e62ae4649f029729...   
3374   https://p.scdn.co/mp3-preview/5477eae2283113ff...   
4319   https://p.scdn.co/mp3-preview/bcaccba847a5cf53...   
4600   https://p.scdn.co/mp3-preview/4ce283ae87d032f5...   
4618   https://p.scdn.co/mp3-preview/e5c557dc2b36006a...   
6047   https://p.scdn.co/mp3-preview/3859547944f57cfb...   
6819   https://p.scdn.co/mp3-preview/bf65095d5ce58358...   
7008   https://p.scdn.co/mp3-preview/d778abba87ff5de6...   
11905  https://p.scdn.co/mp3-preview/8fd280cc8c535862...   
12306  https://p.scdn.co/mp3-preview/d78c90c5cb5626be...   
12323  https://p.scdn.co/mp3-preview/91217265ccb8fc80...   
16861  https://p.scdn.co/mp3-preview/88da75b50a528fe6...   
16947  https://p.scdn.co/mp3-preview/54633474ff7aafb1...   
27044  https://p.scdn.co/mp3-preview/0b112235150f3381...   
29948  https://p.scdn.co/mp3-preview/156365b8cbc625ee...   
32185  https://p.scdn.co/mp3-preview/3d0045ee2fddfb77...   
38389  https://p.scdn.co/mp3-preview/6e2c021846087a88...   
38421  https://p.scdn.co/mp3-preview/762595070b8b966d...   
42435  https://p.scdn.co/mp3-preview/18c4b856a3a5c741...   
42453  https://p.scdn.co/mp3-preview/23892da2ddc26b6c...   
42514  https://p.scdn.co/mp3-preview/63b29a594f6d6976...   
42562  https://p.scdn.co/mp3-preview/28b6b0789340e733...   
43704  https://p.scdn.co/mp3-preview/d6e5a567e639f673...   
43712  https://p.scdn.co/mp3-preview/83185ae92ade51e7...   
43714  https://p.scdn.co/mp3-preview/cc7e2fa060501a1c...   
43735  https://p.scdn.co/mp3-preview/fffbeba5d53973dc...   
43891  https://p.scdn.co/mp3-preview/079500681

In [36]:
# build input vector

song_input = df_content_filtering[df_songs['name'] == "Whenever, Wherever"]
song_input

artist                                               tags  year  \
1025  shakira  rock, pop, female_vocalists, singer_songwriter...  2012   

      duration_ms  danceability  energy  key  loudness  mode  speechiness  \
1025       196826         0.787   0.828    1    -4.967     0       0.0474   

      acousticness  instrumentalness  liveness  valence    tempo  \
1025         0.298          0.000005     0.206     0.86  107.674   

      time_signature  
1025               4

In [37]:
# input vector to calculate similarity

input_vector = transformer.transform(song_input)
input_vector

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 20 stored elements and shape (1, 8431)>

In [38]:
# calculate the similarity matrix

similarity_scores = cosine_similarity(transformed_df, input_vector)
similarity_scores

array([[0.99999914],
       [0.99999847],
       [0.99999921],
       ...,
       [0.99999877],
       [0.9999992 ],
       [0.99999891]])

In [39]:
similarity_scores.shape

(50674, 1)

In [40]:
top_10_songs_indexes = np.argsort(similarity_scores.ravel())[-11:][::-1] # argsort() orts in ascending order (-1 to reverse)
top_10_songs_indexes

array([ 1025, 12305,  6046,  6129, 17241,  6133,  7172,  6121,  6526,
       38383,  6287])

In [41]:
top_10_songs_names = df_content_filtering.loc[top_10_songs_indexes, :]
top_10_songs_names

artist                                               tags  \
1025              shakira  rock, pop, female_vocalists, singer_songwriter...   
12305                beck              rock, alternative, experimental, funk   
6046                 iyaz  pop, dance, rap, hip_hop, rnb, male_vocalists,...   
6129               magic!                        pop, reggae, male_vocalists   
17241           lady gaga                         pop, dance, american, love   
6133   corinne bailey rae  pop, female_vocalists, jazz, singer_songwriter...   
7172      vanessa carlton  pop, female_vocalists, singer_songwriter, pian...   
6121                 nena       pop, female_vocalists, 80s, german, new_wave   
6526          demi lovato                   pop, female_vocalists, beautiful   
38383           steve vai                               instrumental, guitar   
6287             r. kelly              pop, soul, soundtrack, 90s, rnb, love   

       year  duration_ms  danceability  energy  key  loudness  mode  \
1025   2012       196826         0.787   0.828    1    -4.967     0   
12305  1996       215160         0.710   0.886    1    -7.247     1   
6046   2010       181946         0.721   0.746    9    -6.565     1   
6129   2014       223830         0.747   0.752    1    -5.315     1   
17241  2011       316520         0.368   0.833    6    -5.869     0   
6133   2006       240175         0.586   0.222    5   -15.114     1   
7172   2002       238440         0.319   0.671    0    -3.323     1   
6121   2007       234800         0.404   0.433    4   -12.840     1   
6526   2011       220146         0.580   0.539    7    -5.229     1   
38383  1990       351160         0.545   0.770    5    -8.854     1   
6287   2003       320560         0.544   0.473    7    -5.875     1   

       speechiness  acousticness  instrumentalness  liveness  valence  \
1025        0.0474      0.298000          0.000005    0.2060   0.8600   
12305       0.3510      0.050000          0.008180    0.0526   0.5640   
6046        0.0644      0.163000          0.000000    0.1950   0.1820   
6129        0.0354      0.047000          0.000000    0.0703   0.9150   
17241       0.0464      0.000154          0.006510    0.0545   0.4820   
6133        0.0395      0.665000          0.814000    0.0981   0.0792   
7172        0.0329      0.226000          0.000000    0.1690   0.4090   
6121        0.0616      0.093600          0.000005    0.1120   0.5730   
6526        0.0285      0.105000          0.000000    0.1220   0.1980   
38383       0.0474      0.000287          0.750000    0.7180   0.6300   
6287        0.0297      0.163000          0.000000    0.1120   0.0574   

         tempo  time_signature  
1025   107.674               4  
12305  174.460               4  
6046    91.033               4  
6129   144.047               4  
17241  143.982               4  
6133   122.766               4  
7172   185.256               3  
6121   192.711               4  
6526   103.813               4  
38383  127.058               4  
6287   120.459               4

In [42]:
def recommend(song_name, songs_data, transformed_data, k=10):
  """
  Recommends top k songs similar to the given song basedon content-based filtering.

  Parameters:
  song_name (str): the name of the song to base the recommendations on.
  song_data (DataFrame): the dataframe containing information.
  transformed_data (ndarray): the transformed data matrix for similarity calculations.
  k (int, optional): number of similar songs to recommend. default is 10.

  Returns:
  DataFrame: a dataframe containing the top k recommended songs with their names, artists, and Spotify preview URLs.
  """

  # filter out the song from data
  song_row = songs_data.loc[songs_data['name'] == song_name,:]
  if song_row.empty:
    print('Song not found in the dataset')
  else:
    # get the index of the song
    song_index = song_row.index[0]
    print(song_index)

    # generate the input vector
    input_vector = transformed_data[song_index].reshape(1, -1)

    # calculate similarity scores
    similarity_scores = cosine_similarity(input_vector, transformed_data)
    print(similarity_scores.shape)

    # get the top k songs
    top_k_songs_indexes = np.argsort(similarity_scores.ravel())[-k-1:][::-1]
    print(top_10_songs_indexes)

    # get the top k song names
    top_k_songs_names = songs_data.iloc[top_k_songs_indexes]

    # print the top k songs
    top_k_list = top_k_songs_names[['name', 'artist', 'spotify_preview_url']].reset_index(drop=True)

    return top_k_list

In [43]:
# recommend song using the function

recommend("Whenever, Wherever", songs_data=df_songs, transformed_data=transformed_df,k=10)

1025
(1, 50674)
[ 1025 12305  6046  6129 17241  6133  7172  6121  6526 38383  6287]


name           artist  \
0       Whenever, Wherever          Shakira   
1                 Why Wait          Shakira   
2           Hips Don't Lie          Shakira   
3   Oops!...I Did It Again   Britney Spears   
4            Perfect Lover   Britney Spears   
5             Bootylicious  Destiny's Child   
6              Wild Things     Alessia Cara   
7           La Isla Bonita          Madonna   
8             Cruel Summer       Bananarama   
9         Dreams for Plans          Shakira   
10                Trick Me            Kelis   

                                  spotify_preview_url  
0   https://p.scdn.co/mp3-preview/09ddeb4ae33ee6e8...  
1   https://p.scdn.co/mp3-preview/d78c90c5cb5626be...  
2   https://p.scdn.co/mp3-preview/3859547944f57cfb...  
3   https://p.scdn.co/mp3-preview/7fb86827422540ad...  
4   https://p.scdn.co/mp3-preview/52671e54d36f077e...  
5   https://p.scdn.co/mp3-preview/7e327ccb1e4c52b2...  
6   https://p.scdn.co/mp3-preview/c13f00088525d0b2...  
7   https://p.scdn.co/mp3-preview/d8f3cafe99c1f0cd...  
8   https://p.scdn.co/mp3-preview/47d13ef240a58bef...  
9   https://p.scdn.co/mp3-preview/6e2c021846087a88...  
10  https://p.scdn.co/mp3-preview/0862413f9e28d284...